In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load dataset using pandas 

In [ ]:
data =  pd.read_csv('/content/drive/My Drive/Bangla letters/data.csv')

In [ ]:
data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,label
0,1,1,1,1,1,0,1,0,1,1,4,2,1,5,0,2,4,4,4,4,4,1,0,1,1,1,4,4,0,0,1,2,8,7,5,0,1,2,0,0,...,1,1,0,0,0,6,0,138,255,193,0,0,0,0,2,0,1,3,0,1,2,1,2,3,0,1,0,0,0,0,0,0,1,5,1,65,253,123,0,10
1,2,1,36,14,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,7,230,162,14,3,1,0,0,0,0,0,...,2,0,0,141,255,45,18,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6,2,152,245,20,0,3,0,0,0,10
2,0,0,0,0,0,1,0,1,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,42,111,0,3,...,0,0,2,2,1,24,253,153,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,4,0,0,6,1,1,1,4,0,145,59,0,2,0,10
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,6,3,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,...,2,0,0,0,0,0,0,0,0,0,0,1,2,0,53,174,186,204,181,180,69,0,3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,10
4,0,0,0,0,0,1,0,1,5,0,0,0,5,22,19,20,0,0,2,0,0,0,0,0,0,1,1,1,0,0,0,0,2,3,4,0,0,14,52,114,...,5,1,111,255,202,0,2,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,2,0,0,0,4,0,102,248,167,1,3,0,0,0,0,10


In [ ]:
data["label"].unique()

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [ ]:
X = (data.drop(["label"], axis=1))/255.0

In [ ]:
y = data['label']

### Split the data into train and test set using sci-kit learn train_test_split. The use of stratify arguement ensures that both train and test dataset contain all the classes.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

In [ ]:
img_rows = img_cols = 28

In [ ]:
x_train = X_train.values.reshape(X_train.shape[0], img_rows, img_cols, 1)
x_test = X_test.values.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
num_classes = len(y_test.unique())

In [ ]:
len(y_test.unique()),len(y_train.unique())

(50, 50)

### using to_categorical method for One-hot encoding

In [ ]:
import keras
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [ ]:
data['label'].unique()

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

### We are defining the our CNN model here.

In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
batch_size = 128
epochs = 24
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 55997 samples, validate on 18666 samples
Epoch 1/24
55997/55997 [==============================] - 23s 403us/step - loss: 0.4824 - acc: 0.8565 - val_loss: 0.4478 - val_acc: 0.8774
Epoch 2/24
55997/55997 [==============================] - 22s 399us/step - loss: 0.4490 - acc: 0.8645 - val_loss: 0.4471 - val_acc: 0.8795
Epoch 3/24
55997/55997 [==============================] - 22s 401us/step - loss: 0.4311 - acc: 0.8710 - val_loss: 0.4300 - val_acc: 0.8840
Epoch 4/24
55997/55997 [==============================] - 22s 399us/step - loss: 0.4115 - acc: 0.8745 - val_loss: 0.4333 - val_acc: 0.8850
Epoch 5/24
55997/55997 [==============================] - 23s 403us/step - loss: 0.3960 - acc: 0.8802 - val_loss: 0.4292 - val_acc: 0.8892
Epoch 6/24
55997/55997 [==============================] - 23s 417us/step - loss: 0.3774 - acc: 0.8862 - val_loss: 0.4185 - val_acc: 0.8877
Epoch 7/24
55997/55997 [==============================] - 23s 403us/step - loss: 0.3642 - acc: 0.8898 - val_loss: 0.

### Our model has an accuracy of 88.9%. 

In [ ]:
model.save("/content/drive/My Drive/Bangla letters/banglachar.h5")

In [ ]:
'''def predict(image):
    input = cv2.resize(image, (28 , 28)).reshape((28 , 28,1)).astype('float32') / 255
    return model.predict_classes(np.array([input]))'''

In [ ]:
'''# creating a 600 x 600 pixels canvas for mouse drawing
canvas = np.ones((600,600), dtype="uint8") * 255
# designating a 400 x 400 pixels point of interest on which digits will be drawn
canvas[100:500,100:500] = 0

start_point = None
end_point = None
is_drawing = False

def draw_line(img,start_at,end_at):
    cv2.line(img,start_at,end_at,255,15)

def on_mouse_events(event,x,y,flags,params):
    global start_point
    global end_point
    global canvas
    global is_drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        if is_drawing:
            start_point = (x,y)
    elif event == cv2.EVENT_MOUSEMOVE:
        if is_drawing:
            end_point = (x,y)
            draw_line(canvas,start_point,end_point)
            start_point = end_point
    elif event == cv2.EVENT_LBUTTONUP:
        is_drawing = False


cv2.namedWindow("Test Canvas")
cv2.setMouseCallback("Test Canvas", on_mouse_events)


while(True):
    cv2.imshow("Test Canvas", canvas)
    key = cv2.waitKey(1) & 0xFF 
    if key == ord('q'):
        break
    elif key == ord('s'):
        is_drawing = True
    elif key == ord('c'):
        canvas[100:500,100:500] = 0
    elif key == ord('p'):
        image = canvas[100:500,100:500]
        result = predict(image)
        print("PREDICTION : ",result)

cv2.destroyAllWindows()'''